In [1]:
from mpetools import IslandTime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import folium
import webbrowser

In [27]:
path_to_data = os.path.join(os.getcwd(), 'data', 'info_islands')

latitude, longitude, colors, names, desc = [], [], [], [], []

for file in os.listdir(path_to_data):
    island = file.split('_')[1]
    country = file.split('_')[2].split('.')[0]
    island_info = IslandTime.retrieve_island_info(island, country, verbose=False)
    latitude.append(island_info['spatial_reference']['latitude'])
    longitude.append(island_info['spatial_reference']['longitude'])
    names.append(island_info['general_info']['island'])

    if 'visual_inspection' in island_info['general_info'].keys():
        colors.append('blue')
        desc.append('Visual inspection done')
    
    else:
        if 'timeseries_coastsat' in island_info.keys():
            if 'timeseries' in island_info['timeseries_coastsat'].keys():
                colors.append('green')
                desc.append('CoastSat time series available')
            
            else:
                if os.path.exists(os.path.join(os.getcwd(), 'data', 'coastsat_data', island+'_'+country)):
                    colors.append('orange')
                    desc.append('CoastSat images are being downloaded')

        
        else:
            if os.path.exists(os.path.join(os.getcwd(), 'data', 'coastsat_data', island+'_'+country)):
                colors.append('orange')
                desc.append('CoastSat images are being downloaded')
            
            else:
                colors.append('red')
                desc.append('CoastSat has not started downloading images yet')

# Create a base map
m = folium.Map(location=[sum(latitude) / len(latitude), sum(longitude) / len(longitude)], zoom_start=5)

# Add markers for each point
for lat, lon, color, name, des in zip(latitude, longitude, colors, names, desc):
    folium.Marker([lat, lon], popup=str(name)+'\n'+str(des), icon=folium.Icon(color=color)).add_to(m)

# Save the map to an HTML file
m.save('island_mapping.html')

# Open the map in a web browser
webbrowser.open('island_mapping.html')

True